<a href="https://colab.research.google.com/github/VladLoPG/Lobanov-Programming-HSE/blob/main/Automated_opinion_mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Устанавливаем необходимые зависимости

In [ ]:
!pip install pandas tqdm datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 16.5 MB/s eta 0:00:00


# Загрузка ллама с++. Почему-то только с этой версией llama-cpp получается активировать GPU в колабе, иначе работает центральный процессор, а он для этой задачи безумно медленный

In [ ]:
!CMAKE_ARGS="-DGGML_CUDA=on -DCMAKE_CUDA_ARCHITECTURES=75" pip install --no-cache-dir llama-cpp-python==0.2.77 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu122
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.7/315.7 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 13.6 MB/s eta 0:00:00


# Загрузка модели

In [ ]:
from llama_cpp import Llama
from huggingface_hub import hf_hub_download

# Download the model and get the local file path
repo_id = "yandex/YandexGPT-5-Lite-8B-instruct-GGUF"
filename = "YandexGPT-5-Lite-8B-instruct-Q4_K_M.gguf"

# Download the model file and store the path
MODEL_PATH = hf_hub_download(repo_id=repo_id, filename=filename)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


YandexGPT-5-Lite-8B-instruct-Q4_K_M.gguf:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

# Импорт всех нужных библиотек

In [ ]:
import pandas as pd
import re
from tqdm import tqdm
from datetime import datetime
import unicodedata
from collections import defaultdict
import time

# Код для извлечения ключевых слов, результат сохранится в таблицу csv, пополняется по мере обработки отзывов, через каждые 50

In [ ]:
# Пути к файлам
CSV_FILE_PATH = "/content/all_with_google_spaso-efim_df.csv"
OUTPUT_CSV_PATH = "keywords_spaso_efim.csv"

# Параметры модели
N_CTX = 3072 # размер контекста для обработки одного отзыва
N_THREADS = 2 # кол-во ядер цпу
MAX_TOKENS = 128 # максимальное кол-во токенов на ответ модели
TEMPERATURE = 0.1 # температура модели, чем выше, тем рандомнее ответ
MAX_RETRIES = 2  # максимальное кол-во повторных обработок при неудаче обработки отзыва

def load_model(model_path: str, n_threads: int, n_ctx: int):
    """Загрузка модели с использованием GPU, параметры модели определяются в константах в начале кода"""
    try:
        return Llama(
            model_path=model_path,
            n_ctx=n_ctx,       # Размер контекста
            n_gpu_layers=-1,  # Используем GPU (можно -1 для авто)
            n_threads=n_threads,
            verbose=False     # Отключить или True для диагностики
        )
    except Exception as e:
        raise SystemExit(f"Ошибка загрузки модели: {e}")

def extract_year(date_str):
    """Извлечение года из строки даты в разных форматах"""
    if not isinstance(date_str, str) or not date_str.strip():
        return "unknown"
    try:
        dt = pd.to_datetime(date_str, errors='coerce')
        if pd.notna(dt):
            return str(dt.year)

        if "-" in date_str:
             dt_dash = pd.to_datetime(date_str, format='%d-%m-%Y', errors='coerce')
             if pd.notna(dt_dash):
                 return str(dt_dash.year)

        match = re.search(r'\b(\d{4})\b', date_str)
        if match:
            year = int(match.group(1))
            current_year = datetime.now().year
            if 1900 <= year <= current_year:
                 return str(year)

        return "unknown"
    except Exception:
        return "unknown"

def parse_response(response: str) -> dict | None:
    """
    Парсинг ответа модели с использованием строгого регулярного выражения.
    Ожидает формат: {id}, {year}, {positives}, {negatives} (4 элемента)
    Возвращает словарь при успехе, иначе None.
    """
    pattern = r'^\s*\{([^}]*)\}\s*,\s*\{([^}]*)\}\s*,\s*\{([^}]*)\}\s*,\s*\{([^}]*)\}\s*$'
    match = re.match(pattern, response)

    if match:
        review_id, year, positives, negatives = [part.strip() for part in match.groups()]
        if review_id and (year.isdigit() or year == "unknown"):
            return {
                "review_id": review_id,
                "year": year,
                "positives": positives if positives and positives != "-" else "-",
                "negatives": negatives if negatives and negatives != "-" else "-",
                "failed": False
            }
    return None

def analyze_review(llm, review_text: str, rating: int, review_id: str, year: str, max_retries: int = 3) -> dict:
    """Анализ отзыва с проверкой формата"""
    if not isinstance(review_text, str) or not review_text.strip():
        return {
            "review_id": review_id, "year": year,
            "positives": "-", "negatives": "-", "failed": False
        }

    prompt = f"""
Инструкции:
1. Извлекай краткие, полезные положительные и отрицательные ключевые слова/фразы ТОЛЬКО из текста указанного отзыва. Избегай оценочных фраз по типу "прекрасный музей"/"величественный собор". Пиши на русском.
2. Если положительных или отрицательных аспектов в тексте нет, пиши "-".
3. ВСЕГДА возвращай результат СТРОГО в формате: {{review_id}}, {{year}}, {{positive keywords}}, {{negative keywords}}
4. Все 4 элемента должны быть в фигурных скобках и разделены запятыми.
5. Не добавляй никакого другого текста или пояснений до или после этого формата.

Примеры:
Отзыв: "Отличный вид на город, но подъем очень высокий и крутой." Рейтинг: 3
Ответ: {{review_1}}, {{2022}}, {{отличный вид на город}}, {{подъем очень высокий и крутой}}
---
Задание:
Отзыв: {review_text}
Рейтинг: {rating}
review_id: {review_id}
year: {year}

Верни ТОЛЬКО одну строку в указанном формате.
Ответ:
""".strip()

    for attempt in range(max_retries):
        try:
            llm_response = llm(prompt, max_tokens=MAX_TOKENS, temperature=TEMPERATURE, stop=['\n'])

            response_text = ""
            if llm_response and 'choices' in llm_response and llm_response['choices']:
                 response_text = llm_response["choices"][0]["text"].strip()

            if not response_text:
                print(f"\nПопытка {attempt + 1} для {review_id}: Пустой ответ от модели.")
                continue

            parsed_response = parse_response(response_text)
            if parsed_response:
                if parsed_response["review_id"] == review_id and parsed_response["year"] == year:
                    return parsed_response
                else:
                    print(f"\nПопытка {attempt + 1} для {review_id}: ID/Год в ответе не совпали (Ожидали: {review_id}/{year}, Получили: {parsed_response.get('review_id')}/{parsed_response.get('year')}). Ответ: {response_text}")
                    continue
            else:
                print(f"\nПопытка {attempt + 1} для {review_id}: Неверный формат ответа модели. Ответ: {response_text}")
                continue

        except Exception as e:
            print(f"\nОшибка при вызове LLM для {review_id} на попытке {attempt + 1}: {e}")
            continue

    print(f"\nНе удалось обработать {review_id} после {max_retries} попыток.")
    return {
        "review_id": review_id, "year": year,
        "positives": "failed_processing", "negatives": "-",
        "failed": True
    }

def process_csv_reviews(csv_file_path: str, output_csv_path: str):
    """Основная функция, которая последовательно обрабатывает отзывы:
    1. Загружает модель
    2. Читает таблицу с отзывами
    3. Фильтрует пустые отзывы
    4. Собственно обрабатывает отзывы, подгружая по 50 строк из таблицы
    5. Сохраняет результаты обработки в таблицу csv
    """
    try:
        llm = load_model(MODEL_PATH, N_THREADS, N_CTX)
    except SystemExit as e:
        print(e)
        return

    try:
        df = pd.read_csv(csv_file_path)
        required_columns = {"publication_date", "rating", "text"}
        if not required_columns.issubset(df.columns):
            raise ValueError(f"Отсутствуют необходимые столбцы: {required_columns - set(df.columns)}")
    except FileNotFoundError:
        print(f"Ошибка: Файл не найден по пути {csv_file_path}")
        return
    except Exception as e:
        print(f"Ошибка чтения CSV: {e}")
        return

    print("Подготовка данных для обработки (фильтрация пустых отзывов и отзывов вне интересующего периода)...")
    tasks = []
    all_results_map = {}
    empty_review_count = 0
    outdated_review_count = 0

    for index, row in df.iterrows():
        review_text = row.get("text", "")
        review_id = f"review_{index + 1}"
        if pd.isna(review_text) or not str(review_text).strip():
            empty_review_count += 1
            continue

        year = extract_year(row.get("publication_date"))
        if  year == 'unknown' or not 2020 <= int(year) <= 2025:
            outdated_review_count += 1
            continue
        rating = row.get("rating")
        task = (
            str(review_text).strip(),
            rating,
            review_id,
            year,
            MAX_RETRIES,
            MODEL_PATH,
            N_THREADS,
            N_CTX,
            MAX_TOKENS,
            TEMPERATURE
        )
        tasks.append(task)
        all_results_map[review_id] = {
            "review_id": review_id,
            "year": year,
            "status": "pending",
            "positives": "-",
            "negatives": "-"
        }

    print(f"Отфильтровано пустых отзывов: {empty_review_count}")
    print(f"Отфильтровано отзывов, не попавших в промежуток с 2020-2025 год: {outdated_review_count}")
    if not tasks:
        print("Нет данных для обработки.")
        return

    total_tasks = len(tasks)
    print(f"Начинаем анализ {total_tasks} отзывов...")

    results_list = []
    failed_analysis_count = 0
    is_first_write = True  # Flag to control header writing

    for task in tqdm(tasks, total=total_tasks, desc="Обработка отзывов", mininterval=1.0):
        review_text, rating, review_id, year, _, _, _, _, _, _ = task
        analysis_result = analyze_review(llm, review_text, rating, review_id, year, MAX_RETRIES)
        if analysis_result.get("failed", False):
            failed_analysis_count += 1

        output_columns = ['review_id', 'year', 'positives', 'negatives']
        record = {col: analysis_result.get(col, '-') for col in output_columns}
        if record["positives"] == "failed_processing":
            record["positives"] = "-"
        results_list.append(record)

        if len(results_list) >= 50:
            pd.DataFrame(results_list[-50:], columns=output_columns).to_csv(
                output_csv_path,
                mode='w' if is_first_write else 'a',
                index=False,
                encoding='utf-8',
                header=is_first_write
            )
            is_first_write = False
            results_list = results_list[:-50]

    if results_list:
        pd.DataFrame(results_list, columns=output_columns).to_csv(
            output_csv_path,
            mode='w' if is_first_write else 'a',
            index=False,
            encoding='utf-8',
            header=is_first_write
        )

    print("\nАнализ завершен.")
    print(f"Результаты сохранены в '{output_csv_path}'")
    if failed_analysis_count > 0:
        print(f"Количество отзывов, не обработанных LLM после всех попыток: {failed_analysis_count}")

# Запуск
if __name__ == "__main__":
    process_csv_reviews(CSV_FILE_PATH, OUTPUT_CSV_PATH)

# Код для анализа ключевых слов. На вход принимает табличку с ключевыми словами из предыдущего кода, на выходе получаем файл txt с разбивкой по категориям для каждого года с 2020 по 2025

In [ ]:
# Пути к файлам
INPUT_CSV_PATH = "/content/keywords_titan (6).csv"
OUTPUT_TXT_PATH = "/content/reports/report_spaso_efim.txt"
# 2020-2025 включительно

# Model parameters
N_CTX = 32000 # Увеличено для потенциально больших списков слов за год
N_THREADS = 2 # Кол-во ядер ЦПУ
MAX_TOKENS = 3072 # Увеличено для более детальных ответов по категориям
TEMPERATURE = 0.1 # Температура модели
MAX_RETRIES = 3 # Максимальное кол-во повторных попыток при неправильном ответе модели
RETRY_DELAY = 5 # Секунды задержки между повторными попытками
YEAR_RANGE = range(2020, 2026) # Годы, по которым идет анализ

def load_model(model_path: str):
    """Загрузка модели с использованием GPU, параметры модели определяются в константах в начале кода"""
    try:
        return Llama(
            model_path=model_path,
            n_ctx=N_CTX,
            n_gpu_layers=-1, # максимально задействует графический процессор
            n_threads=N_THREADS,
            verbose=False
        )
    except Exception as e:
        raise SystemExit(f"Ошибка загрузки модели: {e}")

def sanitize_text(text: str) -> str:
    """Выполняет простую очистку текста ключевых слов"""
    if pd.isna(text) or not isinstance(text, str) or text.strip() == "-" or not text.strip():
        return ""
    # Нормализация Unicode для единообразия (например, разные типы кавычек)
    text = unicodedata.normalize('NFKC', text)
    # Удаление символов, которые не являются буквами, цифрами, знаками препинания или пробелами
    # Оставляем кириллицу, латиницу, цифры и базовую пунктуацию
    text = re.sub(r'[^\w\s,.!?-]', ' ', text, flags=re.UNICODE)
    # Замена множественных пробелов на один
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def deduplicate_keywords(keywords: list) -> list:
    """Дедуплицирует ключевые слова (удаляет повторяющиеся)"""
    seen = set()
    deduplicated = []
    for kw in keywords:
        # Приводим к нижнему регистру для дедупликации, но сохраняем оригинальный регистр (опционально)
        # Если регистр не важен, можно сразу добавлять .lower() в seen и deduplicated
        kw_lower = kw.lower()
        if kw_lower not in seen:
            seen.add(kw_lower)
            deduplicated.append(kw) # Сохраняем оригинальный регистр
    return deduplicated

def parse_yearly_response(response: str, year: int, total_reviews_for_year: int) -> dict:
    """Парсит ответ модели по каждому году и проверяет на соответствие шаблону"""
    result = {
        "positive_categories": [],
        "negative_categories": []
    }
    response = response.strip()

    # Ищем маркеры секций
    positive_marker = "Позитивные категории"
    negative_marker = "Негативные категории"

    try:
        positive_start_index = response.find(positive_marker)
        negative_start_index = response.find(negative_marker)

        if positive_start_index == -1 or negative_start_index == -1:
            print(f"Ошибка ({year}): Не найдены маркеры '{positive_marker}' или '{negative_marker}'. Ответ: {response[:500]}...")
            return result # Возвращаем пустой результат

        # Извлекаем тексты секций
        positive_section_text = response[positive_start_index + len(positive_marker):negative_start_index].strip()
        negative_section_text = response[negative_start_index + len(negative_marker):].strip()

        def parse_section(section_text: str, section_name: str) -> list:
            '''Парсит ответ модели по положительной/негативной категории и проверяет на соответствие шаблону'''
            categories = []
            lines = [line.strip() for line in section_text.split("\n") if line.strip()]
            for line in lines:
                # Используем regex для более гибкого парсинга строки
                # Формат: Категория - описание: кол-во / общее_кол-во (процент%)
                match = re.match(r'^(.*?)\s*-\s*(.*?):\s*(\d+\s*/\s*\d+\s*\(\s*\d+%?\s*\))$', line)
                if match:
                    category, description, counts = match.groups()
                    categories.append({
                        "category": category.strip(),
                        "description": description.strip(),
                        "counts": counts.strip()
                    })
                # Обработка случая "Нет данных"
                elif "Нет данных" in line and "Отсутствуют релевантные слова" in line:
                     match_nodata = re.match(r'^(.*?)\s*-\s*(.*?):\s*(\d+\s*/\s*\d+\s*\(\s*\d+%?\s*\))$', line)
                     if match_nodata:
                         category, description, counts = match_nodata.groups()
                         categories.append({
                             "category": category.strip(),
                             "description": description.strip(),
                             "counts": counts.strip()
                             })
                     else:
                        print(f"Предупреждение ({year}, {section_name}): Не удалось распарсить строку 'Нет данных': {line}")
                else:
                    print(f"Предупреждение ({year}, {section_name}): Неверный формат строки, пропуск: {line}")
            return categories

        result["positive_categories"] = parse_section(positive_section_text, "Позитивные")
        result["negative_categories"] = parse_section(negative_section_text, "Негативные")

    except Exception as e:
        print(f"Ошибка парсинга ({year}): {e}. Ответ: {response[:500]}...")
        # Возвращаем дефолтные значения при ошибке парсинга
        result = {
            "positive_categories": [{"category": "Ошибка парсинга", "description": "Не удалось обработать ответ модели", "counts": f"0 / {total_reviews_for_year} (0%)"}],
            "negative_categories": [{"category": "Ошибка парсинга", "description": "Не удалось обработать ответ модели", "counts": f"0 / {total_reviews_for_year} (0%)"}]
        }

    # Добавляем категории "Нет данных", если секция пуста и не было ошибки парсинга
    if not result["positive_categories"] and "Ошибка парсинга" not in str(result["positive_categories"]):
         result["positive_categories"].append({"category": "Нет данных", "description": "Отсутствуют релевантные слова", "counts": f"0 / {total_reviews_for_year} (0%)"})
    if not result["negative_categories"] and "Ошибка парсинга" not in str(result["negative_categories"]):
         result["negative_categories"].append({"category": "Нет данных", "description": "Отсутствуют релевантные слова", "counts": f"0 / {total_reviews_for_year} (0%)"})


    return result


def analyze_keywords_for_year(llm, year: int, positive_keywords: list, negative_keywords: list, total_reviews_for_year: int) -> dict:
    """Use the model to categorize keywords for a specific year."""
    print(f"\n--- Анализ для года: {year} ---")
    print(f"Всего отзывов за год: {total_reviews_for_year}")

    # Sanitize keywords (пропускаем пустые строки после санитизации)
    positive_sanitized = [kw for kw in map(sanitize_text, positive_keywords) if kw]
    negative_sanitized = [kw for kw in map(sanitize_text, negative_keywords) if kw]

    # Deduplicate keywords for this year
    positive_deduplicated = deduplicate_keywords(positive_sanitized)
    negative_deduplicated = deduplicate_keywords(negative_sanitized)

    # # Ограничение длины входных данных для промпта (опционально, я не делал, так как их много)
    # # MAX_KW_DISPLAY = 50 # Показать N первых ключевых слов в логе и промпте
    # positive_input = ", ".join(positive_deduplicated) if positive_deduplicated else "нет"
    # negative_input = ", ".join(negative_deduplicated) if negative_deduplicated else "нет"

    # # Добавляем многоточие, если список был обрезан
    # if len(positive_deduplicated) > MAX_KW_DISPLAY: positive_input += "..."
    # if len(negative_deduplicated) > MAX_KW_DISPLAY: negative_input += "..."


    print(f"Позитивные слова ({len(positive_deduplicated)})")
    print(f"Негативные слова ({len(negative_deduplicated)})")

    if not positive_deduplicated and not negative_deduplicated:
        print(f"Нет ключевых слов для анализа за {year} год.")
        return {
            "positive_categories": [{"category": "Нет данных", "description": "Отсутствуют релевантные слова", "counts": f"0 / {total_reviews_for_year} (0%)"}],
            "negative_categories": [{"category": "Нет данных", "description": "Отсутствуют релевантные слова", "counts": f"0 / {total_reviews_for_year} (0%)"}]
        }

    # Используем полный список дедуплицированных слов для промпта, если он не слишком длинный
    # Если слов очень много, можно рассмотреть передачу только части или более агрессивную дедупликацию/фильтрацию
    full_positive_input = ", ".join(positive_deduplicated) if positive_deduplicated else "нет"
    full_negative_input = ", ".join(negative_deduplicated) if negative_deduplicated else "нет"

    prompt = f"""
Инструкции:
1. Проанализируй предоставленные позитивные и негативные ключевые слова из отзывов для {year} года.
2. Сгруппируй похожие по смыслу слова в осмысленные категории.
3. Для каждой категории сформулируй описание, объясняющее её суть. Важно: Описание должно содержать как минимум два словосочетания вида "прилагательное + существительное" (например: "увлекательные лекции", "современное оборудование").
4. Подсчитай количество отзывов, упоминающих слова из каждой категории, и представь это в формате "кол-во_в_категории / {total_reviews_for_year} (процент%)". Процент округляй до целого. Не выделяй категории, для которых всего один отзыв. Выводи категории в порядке убывания количества отзывов.
5. Если для позитива или негатива нет релевантных слов или они не формируют категорию, используй специальную категорию "Нет данных" с описанием "Отсутствуют релевантные слова" и подсчетом "0 / {total_reviews_for_year} (0%)".
6. Выведи результат строго в две секции: "Позитивные категории" и "Негативные категории". Обе секции должны присутствовать, даже если одна из них содержит только "Нет данных".
7. Не добавляй никакой лишний текст: ни введение, ни заключение, ни заголовки года, ни комментарии – только результат в указанном формате.

Примеры Формата и Описаний:

* Пример позитивной категории (с 2+ "прил.+сущ." в описании):
    Экспозиция и Выставки - Интересные экспонаты и познавательные материалы, частые обновления коллекций: 58 / {total_reviews_for_year} (X%)
* Пример другой позитивной категории:
    Атмосфера и Комфорт - Приятная обстановка и чистое помещение, удобные места для отдыха: 29 / {total_reviews_for_year} (Y%)
* Пример негативной категории (с 2+ "прил.+сущ." в описании):
    Работа персонала - Медленное обслуживание и невнимательные сотрудники, долгое ожидание ответа: 21 / {total_reviews_for_year} (Z%)
* Пример категории "Нет данных":
    Нет данных - Отсутствуют релевантные слова: 0 / {total_reviews_for_year} (0%)

Данные для анализа:

Общее количество отзывов за {year} год: {total_reviews_for_year}

Позитивные слова: {full_positive_input}
Негативные слова: {full_negative_input}
""".strip()

    # print(f"Промпт для {year}:\n{prompt}\n--------------------") # Раскомментировать для отладки промпта

    for attempt in range(MAX_RETRIES):
        try:
            print(f"Отправка запроса LLM для года {year} (Попытка {attempt + 1}/{MAX_RETRIES})...")
            llm_response = llm(prompt, max_tokens=MAX_TOKENS, temperature=TEMPERATURE, stop=['\n\n\n']) # Можно добавить больше \n как стоп-токен
            # print(f"LLM Response Raw ({year}, Attempt {attempt+1}): ", llm_response) # Отладка

            if not llm_response or 'choices' not in llm_response or not llm_response['choices']:
                 print(f"Попытка {attempt + 1} ({year}): Некорректный или пустой ответ от LLM.")
                 if attempt < MAX_RETRIES - 1:
                     time.sleep(RETRY_DELAY) # Задержка перед повторной попыткой
                 continue

            response_text = llm_response["choices"][0]["text"].strip()
            # print(f"Текст ответа LLM ({year}, Попытка {attempt+1}):\n{response_text}\n--------------------") # Отладка

            if not response_text:
                print(f"Попытка {attempt + 1} ({year}): Пустой текст ответа от модели.")
                if attempt < MAX_RETRIES - 1:
                     time.sleep(RETRY_DELAY)
                continue

            # Простая проверка наличия ключевых маркеров перед парсингом
            if "Позитивные категории" not in response_text or "Негативные категории" not in response_text:
                 print(f"Попытка {attempt + 1} ({year}): Ответ не содержит ожидаемых секций 'Позитивные категории' или 'Негативные категории'. Ответ: {response_text[:300]}...")
                 if attempt < MAX_RETRIES - 1:
                     time.sleep(RETRY_DELAY)
                 continue


            parsed_response = parse_yearly_response(response_text, year, total_reviews_for_year)

            # Проверяем, что парсинг прошел успешно (есть хотя бы по одной категории или категория "Нет данных"/"Ошибка парсинга")
            if (parsed_response["positive_categories"] or parsed_response["negative_categories"]) and "Ошибка парсинга" not in str(parsed_response["positive_categories"]):
                print(f"Успешно обработан ответ модели для года {year} на попытке {attempt + 1}.")
                return parsed_response
            else:
                 # Логируем ошибку парсинга, если она произошла
                 if "Ошибка парсинга" in str(parsed_response["positive_categories"]):
                    print(f"Попытка {attempt + 1} ({year}): Ошибка во время парсинга ответа.")
                 else:
                    print(f"Попытка {attempt + 1} ({year}): Неверный формат ответа модели после парсинга. Ответ: {response_text[:300]}...")
                 if attempt < MAX_RETRIES - 1:
                    time.sleep(RETRY_DELAY)
                 continue # Повторяем попытку

        except Exception as e:
            print(f"Попытка {attempt + 1} ({year}): Ошибка при вызове или обработке ответа модели: {e}")
            if attempt < MAX_RETRIES - 1:
                 time.sleep(RETRY_DELAY)
            continue # Повторяем попытку

    print(f"Не удалось получить корректный ответ от LLM для года {year} после {MAX_RETRIES} попыток.")
    # Возвращаем структуру с указанием на ошибку
    return {
        "positive_categories": [{"category": "Ошибка LLM", "description": f"Не удалось получить ответ от модели после {MAX_RETRIES} попыток", "counts": f"0 / {total_reviews_for_year} (0%)"}],
        "negative_categories": [{"category": "Ошибка LLM", "description": f"Не удалось получить ответ от модели после {MAX_RETRIES} попыток", "counts": f"0 / {total_reviews_for_year} (0%)"}]
    }

def find_recurring_categories(yearly_results: dict, min_years: int = 3) -> dict:
    """Ищет категории, встречающиеся как минимум 3 года за 2020-2025"""
    positive_category_years = defaultdict(list)
    negative_category_years = defaultdict(list)

    for year, results in yearly_results.items():
        for cat_data in results.get("positive_categories", []):
            cat_name = cat_data.get("category")
            # Игнорируем служебные/ошибочные категории
            if cat_name and cat_name not in ["Нет данных", "Ошибка парсинга", "Ошибка LLM"]:
                positive_category_years[cat_name].append(year)
        for cat_data in results.get("negative_categories", []):
            cat_name = cat_data.get("category")
            if cat_name and cat_name not in ["Нет данных", "Ошибка парсинга", "Ошибка LLM"]:
                negative_category_years[cat_name].append(year)

    recurring = {
        "positive": {cat: years for cat, years in positive_category_years.items() if len(years) >= min_years},
        "negative": {cat: years for cat, years in negative_category_years.items() if len(years) >= min_years}
    }
    # Сортируем словари по имени категории для стабильного вывода
    recurring["positive"] = dict(sorted(recurring["positive"].items()))
    recurring["negative"] = dict(sorted(recurring["negative"].items()))

    return recurring

def process_csv_and_generate_report(csv_file_path: str, output_txt_path: str):
    """Основная функция для обработки ключевых слов
    1. Загружает модель
    2. Читает таблицу с ключевыми словами и переводит в датафрейм
    3. Фильтрует датафрейм по годам
    4. Собственно проводит анализ по годам
    5. Находит повторяющиеся категории
    6. Записывает результат в текстовый файл"""
    try:
        llm = load_model(MODEL_PATH)
    except SystemExit as e:
        print(e)
        return

    try:
        df = pd.read_csv(csv_file_path)
        required_columns = {"publication_date", "positives", "negatives"} # Убедитесь, что эти столбцы есть
        if not required_columns.issubset(df.columns):
            raise ValueError(f"Отсутствуют необходимые столбцы: {required_columns - set(df.columns)}")
        # Преобразуем год в числовой тип, обрабатывая возможные ошибки
        df['publication_date'] = pd.to_numeric(df['publication_date'], errors='coerce')
        df.dropna(subset=['publication_date'], inplace=True) # Удаляем строки с невалидным годом
        df['publication_date'] = df['publication_date'].astype(int)

    except FileNotFoundError:
        print(f"Ошибка: Файл не найден по пути {csv_file_path}")
        return
    except ValueError as e:
         print(f"Ошибка в данных CSV: {e}")
         return
    except Exception as e:
        print(f"Ошибка чтения или обработки CSV: {e}")
        return

    # Фильтруем DataFrame по годам
    df_filtered = df[df['publication_date'].isin(YEAR_RANGE)].copy() # Используем copy() для избежания SettingWithCopyWarning
    if df_filtered.empty:
        print(f"Нет данных для анализа в диапазоне лет {min(YEAR_RANGE)}-{max(YEAR_RANGE)}.")
        return

    # Заполняем пропуски в ключевых словах пустыми строками перед агрегацией
    df_filtered['positives'] = df_filtered['positives'].fillna('')
    df_filtered['negatives'] = df_filtered['negatives'].fillna('')


    total_reviews_overall = len(df_filtered)
    print(f"Загружено и отфильтровано {total_reviews_overall} отзывов за {min(YEAR_RANGE)}-{max(YEAR_RANGE)} годы.")

    yearly_results = {}
    # Группируем по годам и анализируем
    for year, group in df_filtered.groupby('publication_date'):
        if year not in YEAR_RANGE: # Дополнительная проверка
             continue

        total_reviews_for_year = len(group)
        # Собираем все ключевые слова за год. Используем extend для добавления элементов списков.
        # filter(None, ...) убирает пустые строки или None из списков перед передачей в LLM
        positive_keywords_year = list(filter(None, group['positives'].tolist()))
        negative_keywords_year = list(filter(None, group['negatives'].tolist()))

        # Проверяем, есть ли вообще слова для анализа
        if not positive_keywords_year and not negative_keywords_year:
             print(f"Пропуск года {year}: нет позитивных или негативных ключевых слов.")
             # Сохраняем пустой результат, чтобы год присутствовал в отчете
             yearly_results[year] = {
                 "positive_categories": [{"category": "Нет данных", "description": "Отсутствуют исходные ключевые слова", "counts": f"0 / {total_reviews_for_year} (0%)"}],
                 "negative_categories": [{"category": "Нет данных", "description": "Отсутствуют исходные ключевые слова", "counts": f"0 / {total_reviews_for_year} (0%)"}]
             }
             continue


        # Анализ для конкретного года
        result_year = analyze_keywords_for_year(llm, year, positive_keywords_year, negative_keywords_year, total_reviews_for_year)
        yearly_results[year] = result_year
        # Добавим небольшую паузу между запросами к LLM, чтобы не перегружать API (если используется удаленное)
        # или для предотвращения потенциальных проблем с локальной моделью
        time.sleep(1) # Пауза в 1 секунду


    # Находим повторяющиеся категории (минимум 3 года)
    recurring_categories = find_recurring_categories(yearly_results, min_years=3)

    # Записываем результаты в текстовый файл
    print(f"\nЗапись результатов в файл: {output_txt_path}")
    try:
        with open(output_txt_path, 'w', encoding='utf-8') as f:
            f.write(f"Отчет по анализу категорий отзывов за {min(YEAR_RANGE)}-{max(YEAR_RANGE)} года\n")
            f.write(f"Общее количество проанализированных отзывов: {total_reviews_overall}\n")
            f.write("="*40 + "\n\n")

            # Записываем результаты по годам
            for year in sorted(yearly_results.keys()): # Сортируем годы для порядка в отчете
                f.write(f"{year} год\n")
                f.write("-" * 10 + "\n")
                result = yearly_results[year]

                f.write("Позитивные категории\n")
                if result["positive_categories"]:
                    for cat in result["positive_categories"]:
                        f.write(f"{cat['category']} - {cat['description']}: {cat['counts']}\n")
                else:
                     f.write("Нет данных\n") # Если список пуст по какой-то причине

                f.write("\nНегативные категории\n")
                if result["negative_categories"]:
                    for cat in result["negative_categories"]:
                        f.write(f"{cat['category']} - {cat['description']}: {cat['counts']}\n")
                else:
                    f.write("Нет данных\n") # Если список пуст

                f.write("\n" + "="*40 + "\n\n")

            # Записываем повторяющиеся категории
            f.write("Категории, встречающиеся чаще 2-ух лет:\n")
            f.write("-" * 35 + "\n")

            f.write("Позитивные категории:\n")
            if recurring_categories["positive"]:
                for cat, years in recurring_categories["positive"].items():
                    year_str = ", ".join(map(str, sorted(years)))
                    f.write(f"{cat} ({year_str})\n")
            else:
                f.write("Нет повторяющихся позитивных категорий (встречающихся >= 3 раз).\n")

            f.write("\nНегативные категории:\n")
            if recurring_categories["negative"]:
                for cat, years in recurring_categories["negative"].items():
                     year_str = ", ".join(map(str, sorted(years)))
                     f.write(f"{cat} ({year_str})\n")
            else:
                f.write("Нет повторяющихся негативных категорий (встречающихся >= 3 раз).\n")

        print(f"Результаты анализа сохранены в '{output_txt_path}'.")

    except IOError as e:
         print(f"Ошибка записи в файл {output_txt_path}: {e}")
    except Exception as e:
         print(f"Непредвиденная ошибка при записи отчета: {e}")


if __name__ == "__main__":
    process_csv_and_generate_report(INPUT_CSV_PATH, OUTPUT_TXT_PATH)

ggml_backend_cuda_buffer_type_alloc_buffer: allocating 4000.00 MiB on device 0: cudaMalloc failed: out of memory
llama_kv_cache_init: failed to allocate buffer for kv cache
llama_new_context_with_model: llama_kv_cache_init() failed for self-attention cache


Ошибка загрузки модели: Failed to create llama_context


# Код для извлечения ключевых слов из таблицы титан, оставляя информацию о годе, месту жительства посетителя, платформе, никнейму

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ВШЭ/titan_20_25-2-updated.csv')
df.head(2)

,web-scraper-start-url,rating,heading,text,pluses,minuses,name,gender,with_whom,city_where_from,country_where_from,visiting_date,publication_date,object
0,tonkosti,3.0,NaN,Плохая работа персонала Сама экспозиция очень ...,NaN,NaN,kauspeh72,NaN,NaN,NaN,NaN,был(а) здесь в ноябре 2020,2020,derevo_musey
1,otzovik,1.0,Полная хрень,Здравствуйте! Это полная хрень. Развод на день...,Нет,Полно,AnnaFomina0112,NaN,NaN,Краснодар,Россия,NaN,2020,derevo_musey


In [ ]:
df_analysis = df.drop(columns=['pluses', 'minuses', 'rating', 'heading', 'country_where_from', 'with_whom', 'gender'])

In [ ]:
df_analysis['platform'] = df_analysis['web-scraper-start-url']


In [ ]:
df_analysis.drop(columns=['web-scraper-start-url'], inplace=True)
df_analysis.head()

,text,name,city_where_from,visiting_date,publication_date,object,platform
0,Плохая работа персонала Сама экспозиция очень ...,kauspeh72,NaN,был(а) здесь в ноябре 2020,2020,derevo_musey,tonkosti
1,Здравствуйте! Это полная хрень. Развод на день...,AnnaFomina0112,Краснодар,NaN,2020,derevo_musey,otzovik
2,Посещали данный музей в рождество этого года. ...,Иродиада,Владимир,NaN,2020,derevo_musey,otzovik
3,Этим летом проезжали Суздаль и решили посетить...,АНЯМАША,МОСКВА,NaN,2020,derevo_musey,otzovik
4,"Посещали всей семьей этот музей, нам очень пон...",nastyasol1,Москва,NaN,2020,derevo_musey,otzovik


In [ ]:
empty_review_count = 0
for index, row in df_analysis.iterrows():
    review_text = row.get("text", "")
    if pd.isna(review_text) or not str(review_text).strip():
        empty_review_count += 1
        df_analysis.drop(index, inplace=True)
        continue

print(f"Отфильтровано пустых отзывов: {empty_review_count}")
df_analysis.reset_index(drop=True, inplace=True)

Отфильтровано пустых отзывов: 7699


In [ ]:
print(len(df_analysis))
df_analysis[5000:5005]

9995


,text,name,city_where_from,visiting_date,publication_date,object,platform,positives,negatives,gender
5000,"Одно из красивейших мест Суздаля, прекрасно от...",NaN,NaN,NaN,2022,spaso_efim,yandexmaps,,,
5001,На мой взгляд это самый интересный музейный ко...,NaN,NaN,NaN,2022,spaso_efim,yandexmaps,,,
5002,Красивый хорошо сохранившийся монастырь. Его ...,NaN,NaN,NaN,2022,spaso_efim,yandexmaps,,,
5003,Это место обязательно к посещению для туристо...,NaN,NaN,NaN,2022,spaso_efim,yandexmaps,,,
5004,"Отличное место, особенно кто в Суздале в первы...",NaN,NaN,NaN,2022,spaso_efim,yandexmaps,,,


In [ ]:
df_analysis['positives'] = ''
df_analysis['negatives'] = ''
df_analysis['gender'] = ''

In [ ]:
# Пути к файлам
OUTPUT_CSV_PATH = "/content/keywords_titan.csv"

# Model parameters
N_CTX = 4096 # Увеличено для потенциально больших списков слов за год
N_THREADS = 2 # Кол-во ядер ЦПУ
MAX_TOKENS = 128 # Увеличено для более детальных ответов по категориям
TEMPERATURE = 0.1 # Температура модели
MAX_RETRIES = 3 # Максимальное кол-во повторных попыток при неправильном ответе модели

# Объявляем функции для работы

In [ ]:
def load_model(model_path: str, n_threads: int, n_ctx: int):
    """Загрузка модели с использованием GPU, параметры модели определяются в константах в начале кода"""
    try:
        return Llama(
            model_path=model_path,
            n_ctx=n_ctx,       # Размер контекста
            n_gpu_layers=-1,  # Используем GPU (можно -1 для авто)
            n_threads=n_threads,
            verbose=False     # Отключить или True для диагностики
        )
    except Exception as e:
        raise SystemExit(f"Ошибка загрузки модели: {e}")


def parse_response(response: str) -> dict:
  ''' Парсит ответ модели и проверяет его на соответствие формату'''
  try:
    parts = [p.strip() for p in response.split('<sep>')]
    if len(parts) != 3:
      raise ValueError(f"Неверное число полей: {len(parts)}")
    positives, negatives, gender = parts
    if gender in ['М', 'Ж', 'unknown']:
      return {'positives': positives, 'negatives': negatives, 'gender': gender}
    else:
      print(f'Пол не определен: {gender}')
      return
  except Exception as e:
    print(f'Не удалось обработать ответ модели: {e}')
    return


def analyze_review(llm, review_text: str, name: str, max_retries: int = 3) -> dict:
    """Анализ отзыва с проверкой формата"""
    if not isinstance(review_text, str) or not review_text.strip():
        return {
            "review_text": review_text, "name": name,
            "positives": "-", "negatives": "-", "failed": False
        }

    prompt = f"""
Инструкции:
1. Извлекай краткие, полезные положительные и отрицательные ключевые слова/фразы ТОЛЬКО из текста указанного отзыва. Избегай оценочных фраз по типу 'прекрасный музей'/'величественный собор'. Пиши на русском.
2. Если положительных или отрицательных аспектов в тексте нет, пиши -.
3. Предположи пол автора отзыва на основе текста или ника, М для мужчин, Ж для женщин. Если ты не можешь однозначно определить пол, пиши unknown.
3. ВСЕГДА возвращай результат СТРОГО в формате: positive keywords<sep>negative keywords<sep>gender. Если ты хочешь разделить элементы внутри групп, используй запятую
4. Все 3 элемента должны быть разделены токеном <sep>, всего ты можешь использовать только два таких токена.
5. Не добавляй никакого другого текста или пояснений до или после этого формата.

Пример:
Отзыв:
Отличный вид на город, но подъем очень высокий и крутой
AnnaMaslova
Хороший ответ:
отличный вид на город<sep>подъем очень высокий и крутой<sep>Ж
---
Задание:
Текст отзыва: {review_text}
Ник пользователя: {name}

Верни ТОЛЬКО одну строку в указанном формате.
Ответ:
""".strip()

    for attempt in range(max_retries):
        try:
            llm_response = llm(prompt, max_tokens=MAX_TOKENS, temperature=TEMPERATURE)
# ——— DEBUG: show exactly what the model returned
#             print(f"[DEBUG attempt {attempt}] llm_response (type={type(llm_response)}):\n{repr(llm_response)}\n")
            if isinstance(llm_response, dict):
              choices = llm_response.get('choices', [])
              text = choices[0].get('text', '').strip() if choices else ''
            elif isinstance(llm_response, list) and llm_response:
              text = llm_response[0].strip()
            elif isinstance(llm_response, str):
              text = llm_response.strip()
            else:
              text = ''

            if not text:
                print(f"\nПопытка {attempt + 1}: Пустой ответ от модели.")
                continue

            parsed_response = parse_response(text)
            if parsed_response:
              return parsed_response
            else:
              print(f"\nПопытка {attempt + 1}: Неверный формат ответа модели. Ответ: {text}")
              continue

        except Exception as e:
            print(f"\nОшибка при вызове LLM на попытке {attempt + 1}: {e}")
            continue

    print(f"\nНе удалось обработать отзыв после {max_retries} попыток.")
    return {
        "positives": "-", "negatives": "-", "gender": "-",
        "failed": True
    }

def process_csv_reviews(df, output_csv_path: str):
    """Основная функция, которая последовательно обрабатывает отзывы:
    1. Загружает модель
    2. Читает таблицу с отзывами
    3. Фильтрует пустые отзывы
    4. Собственно обрабатывает отзывы, подгружая по 50 строк из таблицы
    5. Сохраняет результаты обработки в таблицу csv
    """
    try:
        llm = load_model(MODEL_PATH, N_THREADS, N_CTX)
    except SystemExit as e:
        print(e)
        return

    try:
        required_columns = {"text", "name"}
        if not required_columns.issubset(df.columns):
            raise ValueError(f"Отсутствуют необходимые столбцы: {required_columns - set(df.columns)}")
    except FileNotFoundError:
        print(f"Ошибка: Файл не найден")
        return
    except Exception as e:
        print(f"Ошибка при чтении датафрейма: {e}")
        return

    tasks = []
    for index, row in df.iterrows():
      review_text = row.get("text", "")
      name = row.get('name', '')
      task = (
          str(review_text).strip(),
          str(name).strip(),
          MAX_RETRIES,
          MODEL_PATH,
          N_THREADS,
          N_CTX,
          MAX_TOKENS,
          TEMPERATURE
      )
      tasks.append(task)

      if not tasks:
          print("Нет данных для обработки.")
          return

    total_tasks = len(tasks)
    print(f"Начинаем анализ {total_tasks} отзывов...")

    results_list = []
    failed_analysis_count = 0

    for i, task in enumerate(tqdm(tasks, total=total_tasks, desc="Обработка отзывов", mininterval=1.0)):
        review_text, name, _, _, _, _, _, _ = task
        analysis_result = analyze_review(llm, review_text, name, MAX_RETRIES)
        if analysis_result.get("failed", False):
            failed_analysis_count += 1

        output_columns = ['positives', 'negatives', 'gender']
        for column in output_columns:
          df.at[i, column] = analysis_result[column]

        if i in range(0, total_tasks, 100): # выводит каждый пятидесятый результат
          print()
          print(f"""Текст отзыва: {df.loc[i]['text']},
Имя пользователя: {df.loc[i]['name']},
Позитивные слова: {df.loc[i]['positives']},
Негативные слова: {df.loc[i]['negatives']},
Предполагаемый пол: {df.loc[i]['gender']}
          """)

    df.to_csv(
        output_csv_path,
        mode='a',
        encoding='utf-8',
        header=True
    )

    print("\nАнализ завершен.")
    print(f"Результаты сохранены в '{output_csv_path}'")
    if failed_analysis_count > 0:
        print(f"Количество отзывов, не обработанных LLM после всех попыток: {failed_analysis_count}")


In [ ]:
process_csv_reviews(df_analysis, OUTPUT_CSV_PATH)

Начинаем анализ 9995 отзывов...


Обработка отзывов:   0%|          | 1/9995 [00:01<4:46:28,  1.72s/it]


Текст отзыва: Плохая работа персонала Сама экспозиция очень интересная...., но общее впечатление испортило отношение персонала, не приветливые уже в начале , на кассе и так по всей территории. На вопросы , которые возникали по ходу просмотра экспозиции отвечали сухо и неохотно..., часто был ответ : Читайте на табличке)). ходили с ощущением, что нам здесь не рады).
05 ноября 2020, 
Имя пользователя: kauspeh72, 
Позитивные слова: интересная экспозиция, 
Негативные слова: плохая работа персонала, не приветливые, сухо и неохотно отвечали, ощущение что не рады, 
Предполагаемый пол: unknown
          


Обработка отзывов:   1%|          | 101/9995 [02:29<3:46:22,  1.37s/it]


Текст отзыва: Сам музей замечательный. Приезжали на праздник огурца, впечатления неоднозначные. Вход 600 р взрослый, дети до 7 лет бесплатно, 400 р льготники, покупайте заранее онлайн. Море народу, сувениры, развлечения - не дешевы: 5 выстрелов из лука 350р, ребёнок выпросил деревянный ножик - 400р,  одна радость - бесплатно дали дождевики от спонсора банка. Концерт хороший, можно потанцевать и поводить хоровод. Народ располагался вокруг сцены на пледах. Пьяных не было. Туалеты чистые, но надо постоять в очереди. Огурцы не покупали, домашние сами солим вкуснее. Если хотите потолкаться и поразевать рот - приезжайте на праздник огурца, но в принципе ничего особенного, только суздальчанам лишний повод подзаработать. PS можете вырастить свой огурец и помериться им в следующем году, или нарядиться в костюм огурца и поучаствовать в конкурсе., 
Имя пользователя: nan, 
Позитивные слова: замечательный музей, 
Негативные слова: вход дорогой, сувениры дорогие, развлечения дорогие, надо постоять 

...


Текст отзыва: Музейная экспозиция, знакомящая посетителей с историей развития жизни на территории Владимирской области, и с фаунистическим и флористическим разнообразием средней полосы европейской части России., 
Имя пользователя: nan, 
Позитивные слова: музейная экспозиция, 
Негативные слова: -, 
Предполагаемый пол: unknown
          


Обработка отзывов:  99%|█████████▉| 9901/9995 [2:16:59<01:01,  1.53it/s]


Текст отзыва: Небольшой музей, хорошие экскурсоводы , 
Имя пользователя: nan, 
Позитивные слова: небольшой музей, хорошие экскурсоводы, 
Негативные слова: -, 
Предполагаемый пол: unknown
          


Обработка отзывов: 100%|██████████| 9995/9995 [2:18:11<00:00,  1.21it/s]


Анализ завершен.
Результаты сохранены в '/content/keywords_titan.csv'
Количество отзывов, не обработанных LLM после всех попыток: 12


## Если часто делаешь дебаг - удаляем модель из кеша, а то будет ругаться

In [ ]:
import gc
import torch

try:
    del llm  # if it's previously defined
except NameError:
    pass

gc.collect()
torch.cuda.empty_cache()


In [ ]:
df_analysis.head()

,text,name,city_where_from,visiting_date,publication_date,object,platform,positives,negatives,gender
0,Плохая работа персонала Сама экспозиция очень ...,kauspeh72,NaN,был(а) здесь в ноябре 2020,2020,derevo_musey,tonkosti,интересная экспозиция,"плохая работа персонала, не приветливые, сухо ...",unknown
1,Здравствуйте! Это полная хрень. Развод на день...,AnnaFomina0112,Краснодар,NaN,2020,derevo_musey,otzovik,-,"полная хрень, развод на деньги, пусто, не инте...",Ж
2,Посещали данный музей в рождество этого года. ...,Иродиада,Владимир,NaN,2020,derevo_musey,otzovik,-,"не радужное впечатление, серо, уныло, два полу...",Ж
3,Этим летом проезжали Суздаль и решили посетить...,АНЯМАША,МОСКВА,NaN,2020,derevo_musey,otzovik,"мастер-класс пошагово все объясняла, время про...",-,Ж
4,"Посещали всей семьей этот музей, нам очень пон...",nastyasol1,Москва,NaN,2020,derevo_musey,otzovik,"приятный персонал, интересные экспонаты, больш...","не так уж и много экспонатов, можно обойти за ...",Ж
